# QOSF Cohort 10
## Task 3

In [343]:
import numpy as np
np.bool = np.bool_
from docplex.mp.model import Model
from qiskit.circuit import QuantumCircuit
from qiskit import transpile 
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import SLSQP
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit.quantum_info.analysis.z2_symmetries import Z2Symmetries
from qiskit_algorithms import VQE
from qiskit.quantum_info import SparsePauliOp
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.cloud import Client
from qiskit.primitives import Estimator
from scipy.optimize import minimize
from qiskit.circuit.library import (
    EfficientSU2,
    RealAmplitudes,
    TwoLocal,
    NLocal,
    ExcitationPreserving
)

1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)

* Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 



The ILP formulation of the BPP is : 

**Variables :**

* binary variables : $x_{i,j}=1$ if item i is placed in bin j, 0 otherwise
* binary variables : $y_j=1$ if bin j is used 0 otherwise

**Objective Function :**

* minimize $\sum_j y_j$

**Constraints:**

* $\forall i, j, x_{i,j}, y_j \in \left\{0,1\right\} $
* $\sum_jx_{i,j}=1, \forall i$ : each item must be placed in exactly one bin
* $\sum_iw_ix_{i,j}\leq Wy_j, \forall j$ : The total weight of items in each bin must not exceed the bin capacity W


In [346]:
# The ILP formultation is
# docplex model
from docplex.mp.model import Model

def create_ilp(weights, capacity):
    # Initialize the model
    model = Model('bin_packing')
    
    num_items = len(weights)
    # Create binary variables for items in bins
    x = model.binary_var_matrix(num_items, num_items, name='x')  # x[i][j] = 1 if item i is in bin j
    y = model.binary_var_list(num_items, name='y')  # y[j] = 1 if bin j is used

    # Constraint 1: Each item must be placed in exactly one bin
    for i in range(num_items):
        model.add_constraint(model.sum(x[i, j] for j in range(num_items)) == 1)

    # Constraint 2: The total weight in each bin must not exceed the capacity
    for j in range(num_items):
        model.add_constraint(model.sum(weights[i] * x[i, j] for i in range(num_items)) <= capacity * y[j])
    # Objective: Minimize the number of bins used
    model.minimize(model.sum(y[j] for j in range(num_items)))

    return model


* Create a function to transform the ILP model into a QUBO 

To Transform the ILP model into a QUBO, we need to express the objaectived functions and the constraint in a quadratic form. To do so we keep the objective functions and reformulate constraints in term of pernalties with slack variables : 

Cosntraint : 
$$Q(x,y) = \sum_j y_j + \sum_{i,j}\left(1-\sum_jx_{i,j}\right)^2+\sum_jmax\left(0,\sum_iw_ix_{i,j}-Wy_j\right)^2$$

Objective 

In [347]:
def ilp_to_qubo(weights, capacity):
    num_items = len(weights)
    num_bins = num_items  # Each item can have its own bin

    # Create QUBO matrix
    qubo_matrix = np.zeros((num_items + num_bins, num_items + num_bins))

    # Objective: Minimize the number of bins used
    for j in range(num_bins):
        qubo_matrix[num_items + j, num_items + j] += 1

    # Constraint: Each item must be assigned to at most one bin
    for i in range(num_items):
        for j in range(num_bins):
            qubo_matrix[i, i] += 1  # Self-penalty for item i
            qubo_matrix[i, num_items + j] -= 2  # Coupling between item and bin

    # Constraint: Total weight in each bin should not exceed capacity
    for j in range(num_bins):
        for i in range(num_items):
            qubo_matrix[num_items + j, num_items + j] += weights[i]**2  # Penalty for excess weight
            qubo_matrix[i, num_items + j] -= weights[i]  # Coupling term

        # Add capacity constraint penalties
        qubo_matrix[num_items + j, num_items + j] -= capacity  # Shift penalty for exceeding capacity

    return qubo_matrix

* Test your function with specific instances (size small, medium, and big)

* Small Instance : 3 items with a maximum capacity of 4
* Medium Instance : 5 items and maximum capacity of 10 
* LArge instance : 7 items and maximum capacity of 15

In [348]:
test_cases = {
    "Small": ([1, 2, 3], 4),               # 3 items, capacity 4
    "Medium": ([2, 5, 3, 7, 1], 10),      # 5 items, capacity 10
    "Large": ([1, 2, 3, 4, 5, 6, 7], 15)  # 7 items, capacity 15
}

# Run tests
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    print("QUBO Matrix:")
    print(qubo_matrix)


Testing Small Instance:
QUBO Matrix:
[[ 3.  0.  0. -3. -3. -3.]
 [ 0.  3.  0. -4. -4. -4.]
 [ 0.  0.  3. -5. -5. -5.]
 [ 0.  0.  0. 11.  0.  0.]
 [ 0.  0.  0.  0. 11.  0.]
 [ 0.  0.  0.  0.  0. 11.]]

Testing Medium Instance:
QUBO Matrix:
[[ 5.  0.  0.  0.  0. -4. -4. -4. -4. -4.]
 [ 0.  5.  0.  0.  0. -7. -7. -7. -7. -7.]
 [ 0.  0.  5.  0.  0. -5. -5. -5. -5. -5.]
 [ 0.  0.  0.  5.  0. -9. -9. -9. -9. -9.]
 [ 0.  0.  0.  0.  5. -3. -3. -3. -3. -3.]
 [ 0.  0.  0.  0.  0. 79.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 79.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 79.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 79.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 79.]]

Testing Large Instance:
QUBO Matrix:
[[  7.   0.   0.   0.   0.   0.   0.  -3.  -3.  -3.  -3.  -3.  -3.  -3.]
 [  0.   7.   0.   0.   0.   0.   0.  -4.  -4.  -4.  -4.  -4.  -4.  -4.]
 [  0.   0.   7.   0.   0.   0.   0.  -5.  -5.  -5.  -5.  -5.  -5.  -5.]
 [  0.   0.   0.   7.   0.   0.   0.  -6.  -6.  -6.  -6.  -6.  -6.  -6.]

2. Create a Brute Force solver for the QUBO problem and solve the specific instances

The brute force solver works in the following way : 
* we go through all possible assignment where each items can go into any of the available bins
* we keep track of the total weight in each bin and ccheck that the total weight does not exceed the capacity, if it does the assignment is considered as not valid and it is skipped
* the optimal solutions is chosen amond the valid assignment by picking the one that uses the less bin
* the retuern is the best assignment and the number of bins used

In [349]:
import itertools

def brute_force_qubo(weights, capacity):
    num_items = len(weights)
    best_solution = None
    best_bin_count = float('inf')

    # Generate all possible assignments of items to bins
    for assignment in itertools.product(range(num_items), repeat=num_items):
        # Initialize total weights for each bin
        total_weights = [0] * num_items
        # Use a set to track which bins are used
        bins_used = set()

        # Calculate total weight per bin and track used bins
        for item_index, bin_index in enumerate(assignment):
            total_weights[bin_index] += weights[item_index]
            bins_used.add(bin_index)  # Mark bin as used

        # Check if any bin exceeds the capacity
        if all(weight <= capacity for weight in total_weights):
            current_bin_count = len(bins_used)
            # Update best solution if this one uses fewer bins
            if current_bin_count < best_bin_count:
                best_bin_count = current_bin_count
                best_solution = assignment

    return best_solution, best_bin_count

In [350]:

# Run brute force solver for each instance
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    solution, bin_count = brute_force_qubo(weights, capacity)
    print(f"Best assignment: {solution}")
    print(f"Minimum number of bins used: {bin_count}")


Testing Small Instance:
Best assignment: (0, 0, 1)
Minimum number of bins used: 2

Testing Medium Instance:
Best assignment: (0, 0, 0, 1, 1)
Minimum number of bins used: 2

Testing Large Instance:
Best assignment: (0, 0, 0, 0, 0, 1, 1)
Minimum number of bins used: 2


To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework. Here is an example.

In [351]:
def solve_qubo(weights, capacity):
    qubo_matrix = ilp_to_qubo(weights, capacity)
    
   # Convert QUBO matrix to a dictionary
    qubo_dict = {(i, j): qubo_matrix[i, j] for i in range(len(qubo_matrix)) for j in range(len(qubo_matrix)) if qubo_matrix[i, j] != 0}

    
    try:
        client = Client.from_config()
    except ValueError as e:
        print("Error:", e)
        print("Make sure you have your API token set up as an environment variable.")
        return
    
    # Set up the D-Wave sampler
    sampler = EmbeddingComposite(DWaveSampler())
    initial_state = {i: 1 for i in range(len(qubo_matrix))}
    # Solve the QUBO problem
    sampleset = sampler.sample_qubo(qubo_dict, num_reads=200)#,initial_state=initial_state)

    return sampleset

In [352]:
# Run quantum annealing simulator for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    solution = solve_qubo(weights, capacity)
    print("Quantum Annealing Solution:")
    print(solution)
    # Get the best sample (the one with the lowest energy)
    best_sample = solution.first.sample
    best_energy = solution.first.energy

    print("Best Sample (solution):", best_sample)
    print("Best Energy:", best_energy)

    # Interpret the best sample
    assigned_bins = {}
    for i, assigned in best_sample.items():
        if assigned not in assigned_bins:
            assigned_bins[assigned] = []
        assigned_bins[assigned].append(i)  # Map item indices to their assigned bins

    # Display the bin assignments
    for bin_id, items in assigned_bins.items():
        print(f"Bin {bin_id}: Items {items}")


Testing Small Instance:
Quantum Annealing Solution:
   0  1  2  3  4  5 energy num_oc. chain_.
0  0  0  0  0  0  0    0.0     200     0.0
['BINARY', 1 rows, 200 samples, 6 variables]
Best Sample (solution): {0: np.int8(0), 1: np.int8(0), 2: np.int8(0), 3: np.int8(0), 4: np.int8(0), 5: np.int8(0)}
Best Energy: 0.0
Bin 0: Items [0, 1, 2, 3, 4, 5]

Testing Medium Instance:
Quantum Annealing Solution:
    0  1  2  3  4  5  6  7  8  9 energy num_oc. chain_.
0   0  0  0  0  0  0  0  0  0  0    0.0     154     0.0
1   0  1  0  0  0  0  0  0  0  0    5.0       3     0.0
2   0  0  0  0  1  0  0  0  0  0    5.0      15     0.0
3   0  0  0  1  0  0  0  0  0  0    5.0       5     0.0
4   0  0  1  0  0  0  0  0  0  0    5.0       7     0.0
5   1  0  0  0  0  0  0  0  0  0    5.0      10     0.0
6   0  0  1  0  1  0  0  0  0  0   10.0       1     0.0
7   0  1  1  0  0  0  0  0  0  0   10.0       1     0.0
8   1  0  0  0  1  0  0  0  0  0   10.0       1     0.0
9   1  0  1  0  0  0  0  0  0  0   10.

Use a Quantum Variational approach to solve the QUBO. 

* Create multiple Ansantz for tests. 
* Build a function with input being the QUBO and Ansantz. Using a hybrid approach solved the QUBO. 

I have used Qiskit framework

For the ansazt, I have used : 
* EfficientSU2 (Linear): A parameterized circuit with linear entanglement.

* EfficientSU2 (Full): Similar to the previous but with full entanglement.

* RealAmplitudes (2 reps): A versatile ansatz with two repetitions.

* TwoLocal (CZ): Alternates between rotations and controlled-Z gates.

* ExcitationPreserving: A circuit that preserves the excitation structure, useful for certain types of problems.


In [353]:
# create multiple ansazts to test
def create_ansatze(num_qubits):
    ansatze = []
    
    # Ansatz 1: EfficientSU2 with linear entanglement
    ansatze.append(EfficientSU2(num_qubits, entanglement='linear', name='EfficientSU2 (Linear)'))
    
    # Ansatz 2: EfficientSU2 with full entanglement
    ansatze.append(EfficientSU2(num_qubits, entanglement='full', name='EfficientSU2 (Full)'))
    
    # Ansatz 3: RealAmplitudes with 2 repetitions
    ansatze.append(RealAmplitudes(num_qubits, reps=2, name='RealAmplitudes (2 reps)'))
    
    # Ansatz 4: TwoLocal with alternating layers of X and Z rotations
    ansatze.append(TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=2, name='TwoLocal (CZ)'))
    
    # Ansatz 5: ExcitationPreserving with full entanglement
    ansatze.append(ExcitationPreserving(num_qubits, name='ExcitationPreserving'))
    
    return ansatze

In [354]:
# hybrid approach to solve the QUBO and a simulator
def variational_solver(qubo_matrix, ansatz, num_qubits):
    # Create a quantum instance
    estimator = Estimator(options={"shots": 2048})
    # Setup VQE
    vqe = VQE(estimator ,ansatz, optimizer=SLSQP())

    # Define the Hamiltonian based on the QUBO
    hamiltonian = qubo_to_hamiltonian(qubo_matrix)

    # Run VQE
    result = vqe.compute_minimum_eigenvalue(hamiltonian)

    return result

In [355]:
def qubo_to_hamiltonian(qubo_matrix):
    num_vars = qubo_matrix.shape[0]
    # Start with the identity operator in the Pauli basis
    hamiltonian = SparsePauliOp.from_list([('I' * num_vars, 0)])  # Identity for num_vars qubits

    for i in range(num_vars):
        # Handle diagonal terms (linear terms)
        if qubo_matrix[i, i] != 0:
            # Create Pauli string with Z on qubit i and I on others
            pauli_string = 'I' * i + 'Z' + 'I' * (num_vars - i - 1)
            hamiltonian += qubo_matrix[i, i] * SparsePauliOp.from_list([(pauli_string, 1)])

        for j in range(i + 1, num_vars):  # Iterate only over upper triangle
            coeff = qubo_matrix[i, j]
            if coeff != 0:
                # Create Pauli string with Z on qubit i and j, and I on others
                pauli_string = 'I' * i + 'Z' + 'I' * (j - i - 1) + 'Z' + 'I' * (num_vars - j - 1)
                hamiltonian += coeff * SparsePauliOp.from_list([(pauli_string, 1)])

    return hamiltonian

In [356]:
def interpret_results(result, weights, capacity):
    optimal_state = result.optimal_point  # Get optimal parameters

    num_items = len(weights)
    num_bins = num_items
    bins = [[] for _ in range(num_bins)]
    bin_weights = [0] * num_bins

    for i in range(num_items):
        for j in range(num_bins):
            if optimal_state[i] > 0.5 and bin_weights[j] + weights[i] <= capacity:
                bins[j].append(i)
                bin_weights[j] += weights[i]
                break

    used_bins = [bin for bin in bins if bin]

    print("Items allocated to bins:")
    for bin_index, items in enumerate(used_bins):
        print(f"Bin {bin_index + 1}: Items {items} (Weights: {[weights[i] for i in items]})")

    print(f"\nTotal number of bins used: {len(used_bins)}")

In [ ]:
# Run VQE for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    ansatzes = create_ansatze(qubo_matrix.shape[0])
    results = {}
    for ansazt in ansatzes:
        print(f"\nTesting Ansatz: {ansazt.name}")
        result = variational_solver(qubo_matrix, ansazt, qubo_matrix.shape[0])
        optimal_value = result.eigenvalue
        print("Optimal value:", optimal_value)
        # Interpret and display the results
        interpret_results(result, weights, capacity)
        #store value for best ansazt
        results[ansazt.name] = optimal_value
    best_ansatz = min(results, key=results.get)
    lowest_value = results[best_ansatz]
    print("\nAll Results per Ansatz:")
    for name, value in results.items():
        if name == best_ansatz:
            print(f"{name}: {value} (Best)")
        else:
            print(f"{name}: {value}")
    


Testing Small Instance:

Testing Ansatz: EfficientSU2 (Linear)


C:\Users\jerom\AppData\Local\Temp\ipykernel_69800\3967288211.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 2048})
C:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimal value: 4.112027849479483
Items allocated to bins:
Bin 1: Items [1] (Weights: [2])
Bin 2: Items [2] (Weights: [3])

Total number of bins used: 2

Testing Ansatz: EfficientSU2 (Full)


C:\Users\jerom\AppData\Local\Temp\ipykernel_69800\3967288211.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 2048})


Optimal value: -5.655317225910568
Items allocated to bins:
Bin 1: Items [0, 2] (Weights: [1, 3])

Total number of bins used: 1

Testing Ansatz: RealAmplitudes (2 reps)


...

C:\Users\jerom\AppData\Local\Temp\ipykernel_69800\3967288211.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 2048})
C:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimal value: 17.360440826072054
Items allocated to bins:
Bin 1: Items [2] (Weights: [3])

Total number of bins used: 1

Testing Ansatz: RealAmplitudes (2 reps)


C:\Users\jerom\AppData\Local\Temp\ipykernel_69800\3967288211.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 2048})
C:\Users\jerom\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Optimal value: 40.26779842257382
Items allocated to bins:
Bin 1: Items [2] (Weights: [3])

Total number of bins used: 1

Testing Ansatz: TwoLocal (CZ)


C:\Users\jerom\AppData\Local\Temp\ipykernel_69800\3967288211.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 2048})


## Use QAOA to solve the QUBO. 

* Create from scratch a QAOA function


In [ ]:
def create_qaoa_circuit(num_qubits, p, gamma, beta):
    # Create a quantum circuit with the required number of qubits
    qc = QuantumCircuit(num_qubits)

    # Apply the Hadamard gate to all qubits
    qc.h(range(num_qubits))

    # Apply the QAOA layers
    for layer in range(p):
        # Apply the problem unitary
        for i in range(num_qubits):
            qc.rz(2 * gamma[layer], i)  # Rotation around Z-axis
            if i < num_qubits - 1:
                qc.cx(i, i + 1)  # CNOT between adjacent qubits

        # Apply the mixer unitary
        for i in range(num_qubits):
            qc.rx(2 * beta[layer], i)  # Rotation around X-axis
    qc.measure_all()
    return qc

In [ ]:
def expectation_value(transpiled_qc, qubo_matrix, backend):
    # Simulate the circuit to get the counts
    job = backend.run(transpiled_qc, shots=2048)
    result = job.result()
    counts = result.get_counts()  # Just call get_counts() without parameters
    expectation = 0

    # Calculate the expectation value based on counts
    total_counts = sum(counts.values())
    for outcome, count in counts.items():
        outcome_value = 0
        for i in range(len(outcome)):
            if outcome[i] == '1':
                outcome_value += 1  # Adjust based on your QUBO matrix
        expectation += count * outcome_value

    return expectation / total_counts if total_counts > 0 else 0

In [ ]:
def optimize_qaoa(qubo_matrix, p):
    num_qubits = qubo_matrix.shape[0]
    backend = Aer.get_backend('aer_simulator')

    # Define parameters for the QAOA circuit
    gamma = [Parameter(f'γ_{i}') for i in range(p)]
    beta = [Parameter(f'β_{i}') for i in range(p)]

    # Objective function to minimize
    def objective_function(params):
        gammas = params[:p]
        betas = params[p:]
        qc = create_qaoa_circuit(num_qubits, p, gammas, betas)
        transpiled_qc = transpile(qc,backend)
        return expectation_value(transpiled_qc, qubo_matrix,backend)

    # Initial guess for gamma and beta
    initial_params = np.random.rand(2 * p)

    # Minimize the objective function
    result = minimize(objective_function, initial_params, method='COBYLA')

    return result

In [ ]:
# Run QAOA for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    p = 1  # Number of layers in QAOA
    result = optimize_qaoa(qubo_matrix, p)
    print("Optimal parameters:", result.x)
    print("Minimum value:", result.fun)

In [ ]:
# Create a simple quantum circuit
simple_qc = QuantumCircuit(1, 1)  # 1 qubit, 1 classical bit
simple_qc.h(0)  # Apply Hadamard gate
simple_qc.measure(0, 0)  # Measure qubit to classical bit

# Set up the backend
backend = Aer.get_backend('aer_simulator')

# Transpile the circuit for the backend
transpiled_qc = transpile(simple_qc, backend)

# Run the circuit using the backend
job = backend.run(transpiled_qc, shots=1024)  # Run with 1024 shots
result = job.result()

# Get counts
counts = result.get_counts(transpiled_qc)
print("Counts:", counts)

# Compare results 

The VQE performed well for to solve the BPP problem as long as we test with several ansatz. For small instance, one ansazt was able to obtain the same solution as the brut force (RealAmplitudes (2 reps)). With this ansazt we have the minimal value for the objective function and the same answer as the brut force. All the other ansazt were not able to find the minimum. For other size the best results is obtained for other ansazt

the quantum annealing approach is dependant of the starting point.


## benchmark : brut force : 
small Instance : 3 items 

2 bins : [0,1] in bin 0 and 2 in bin 1

Medium Instance : 5 items

2 bins : [0,1,2] in bin 0 and [3,4] in bin 1

Large Instance : 7 instances

## VQE (Qiskit) : 
small instance : 3 items

2 bins : [0,1] in bin 0 and 2 in bin 1

medium instance : 5 items



